### Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

In [2]:
imp=r"C:\Users\bosse\Desktop\Notebooks\Data\Project\Kaggle_dedub.csv"
df=pd.read_csv(imp,index_col='id')
df.drop(columns='Unnamed: 0',inplace=True)
display(df.head())
display(df.info())

,currency,backers_count,country,status,usd_pledged,sub_category,goal_usd,city,launched_year,duration_d,creator_projects
id,,,,,,,,,,,
1714728788,CAD,16,CA,failed,577.902078,food,785.82065,Victoria,2018,59.958333,1
1639965256,USD,7,US,successful,211.000000,publishing,93.00000,San Diego,2014,24.000000,1
1582062943,USD,99,US,successful,4697.000000,food,1500.00000,Philadelphia,2020,30.000000,1
1888984372,USD,3,US,failed,222.000000,technology,25000.00000,Chicago,2019,60.000000,1
1409770586,USD,100,US,successful,5410.000000,publishing,5000.00000,Madrid,2015,30.000000,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 180677 entries, 1714728788 to 1126079486
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   currency          180677 non-null  object 
 1   backers_count     180677 non-null  int64  
 2   country           180677 non-null  object 
 3   status            180677 non-null  object 
 4   usd_pledged       180677 non-null  float64
 5   sub_category      180677 non-null  object 
 6   goal_usd          180677 non-null  float64
 7   city              180677 non-null  object 
 8   launched_year     180677 non-null  int64  
 9   duration_d        180677 non-null  float64
 10  creator_projects  180677 non-null  object 
dtypes: float64(3), int64(2), object(6)
memory usage: 16.5+ MB


None

In [3]:
# Potential Simplification of variables
df['country'].replace(['GB','CA'],'GB,CA',inplace=True)
df['country'].replace(['DE','FR','IT','ES','NL','SE','DK','CH','IE','BE','AT','NO','LU','PL','GR','SI'],'Europe',inplace=True)
df['country'].replace(['HK','SG','JP'],'Asia',inplace=True)
df['country'].replace(['MX'],'South America and Mexico',inplace=True)
df['country'].replace(['AU','NZ'],'Australia and new Zealand',inplace=True)
df['sub_category'].replace(['music','film & video','games','comics','dance'],'Entertainment',inplace=True)
df['sub_category'].replace(['art','fashion','design','photography','theater'],'Culture',inplace=True)
df['sub_category'].replace(['technology','publishing','food','crafts','journalism'],'Others',inplace=True)
df['currency'].replace(['AUD','MXN','SEK','HKD','NZD','DKK','SGD','CHF','JPY','NOK','PLN'],'Others',inplace=True)

### Split of the data

In [4]:
# Target Variable
df['status'].replace(['successful','failed'],[1,0],inplace=True)
df_y=df['status']

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns='status'),df_y,test_size=0.2,random_state=42)

In [6]:
# Chosen Variables
not_used_list=['city', 'usd_pledged','backers_count']
num_list=['goal_usd','duration_d']
cat_list=['country','currency','sub_category','creator_projects']

### Preprocessing of the data

In [7]:
# standardizing of the numerical variables
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
# 
X_train_num=X_train[num_list]
X_test_num=X_test[num_list]
X_train_num[num_list]=sc.fit_transform(X_train[num_list])
X_test_num[num_list]=sc.fit_transform(X_test[num_list])

C:\Users\bosse\AppData\Local\Temp\ipykernel_13704\1827690632.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_num[num_list]=sc.fit_transform(X_train[num_list])
C:\Users\bosse\AppData\Local\Temp\ipykernel_13704\1827690632.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_num[num_list]=sc.fit_transform(X_test[num_list])


In [8]:
# OneHotEncoding of the categorical variable of X_train
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)
ohe_train=ohe.fit_transform(X_train[cat_list])
column_name=ohe.get_feature_names_out(cat_list)
X_train_ohe=pd.DataFrame(ohe_train,columns=column_name,index=X_train.index)

In [9]:
# OHE X_test variable
ohe2=OneHotEncoder(sparse=False)
ohe_test=ohe2.fit_transform(X_test[cat_list])
column_name2=ohe2.get_feature_names_out(cat_list)
X_test_ohe=pd.DataFrame(ohe_test,columns=column_name2,index=X_test.index)

In [10]:
len(X_train['city'].value_counts())
# The variable city contain over 11000 different entries, ohe possible with so many variables?
# For the time being, city variable will be dropped

11439

In [11]:
X_train_merge_org=pd.concat([X_train_num,X_train_ohe],axis=1)
X_test_merge=pd.concat([X_test_num,X_test_ohe],axis=1)
X_train_merge_org

,goal_usd,duration_d,country_Asia,country_Australia and new Zealand,country_Europe,"country_GB,CA",country_South America and Mexico,country_US,currency_CAD,currency_EUR,...,currency_Others,currency_USD,sub_category_Culture,sub_category_Entertainment,sub_category_Others,creator_projects_1,creator_projects_16-30,creator_projects_2-5,creator_projects_30+,creator_projects_6-15
id,,,,,,,,,,,,,,,,,,,,,
951016356,-0.012969,-0.244914,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
242521970,-0.034624,-0.244914,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1091631676,-0.028186,0.248514,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1004851582,-0.026396,-0.141988,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2014286258,-0.026636,0.580893,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063338383,-0.034900,-0.244914,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1253472741,-0.022187,-0.244914,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
461455322,-0.033529,1.323924,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


#### Resampling and Analysisvariable

In [12]:
from imblearn.over_sampling import RandomOverSampler 
rOs =RandomOverSampler()
X_over,y_over =rOs.fit_resample(X_train_merge_org,y_train)
len(X_over)

168246

In [13]:
from imblearn.under_sampling import RandomUnderSampler
rUs=RandomUnderSampler()
x_under,y_under=rUs.fit_resample(X_train_merge_org,y_train)
len(x_under)

120836

In [14]:
# Variable for the Analysis
X_train_merge=X_train_merge_org
y_train=y_train

### Modelling of the data

Which model to chose?
- Classification. tree or regression?

Which metric to evaluate?
- Eigentlich ja nur Modelle mit den Daten füttern Verbindung zum Project eher unwichtig, keine Projektdaten



In [15]:
# Maschine learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
X_train_merge.drop(['country_SI','currency_PLN'],axis=1,inplace=True)

KeyError: "['country_SI', 'currency_PLN'] not found in axis"

In [ ]:
# Classification tree
tree=DecisionTreeClassifier()
tree.fit(X_train_merge,y_train)
tree_pred=tree.predict(X_test_merge)
print('Decision Tree R2 value:',tree.score(X_test_merge,y_test))

In [ ]:
# Logistical Regression
log=LogisticRegression()
log.fit(X_train_merge,y_train)
log_pred=log.predict(X_test_merge)
print('Logistical Regression R2 value:',log.score(X_test_merge,y_test))

In [ ]:
# Crosstab
print('Decision Tree')
display(pd.crosstab(y_test,tree_pred,normalize=True, rownames=['True'], colnames=['Prediction']))
print('Logistical Regression')
display(pd.crosstab(y_test,log_pred,normalize=True, rownames=['True'], colnames=['Prediction']))

In [ ]:
# Classification Report
from sklearn.metrics import classification_report
print('Decision Tree \n',classification_report(y_test,tree_pred))
print('Logistical Regression \n',classification_report(y_test,log_pred))

In [ ]:
df_y.value_counts(normalize=True)

In [ ]:
# Conclusion
print('we can see that the logistical regresson has the better overall value with a F1 Score of 0.9, it interesting that the decision tree has significant lower values for false positives')

In [ ]:
# Buildung the Coefficient DataFrame
log_importance=pd.DataFrame({'Variables':X_train_merge.columns,'Coefficient':log.coef_[0]})
log_importance_negatives=log_importance.sort_values('Coefficient',ascending=True)
log_importance_positives=log_importance.sort_values('Coefficient',ascending=False)
display(log_importance_positives)

In [ ]:
display(log_importance_negatives)

In [ ]:
# Plotting the Coefficeint DataFrame
plt.figure(figsize=[10,10])
plt.plot(log_importance_negatives['Variables'][0:5],log_importance_negatives['Coefficient'][0:5])
plt.ylabel('Coefficient')
plt.xlabel('Variables')
plt.title('Top 5 negative Coefficients');
plt.show()
plt.figure(figsize=[10,10])
plt.plot(log_importance_positives['Variables'][0:5],log_importance_positives['Coefficient'][0:5])
plt.ylabel('Coefficient')
plt.xlabel('Variables')
plt.title('Top 5 positives Coefficients');

#### Tests